In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [5]:
label_map = {6:0, 7:1, 8:2, 9:3}
class_names = ['frog', 'horse', 'ship', 'truck']
cifar2 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [6, 7, 8, 9]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [6, 7, 8, 9]]

In [6]:
import torch.nn as nn

n_out = 4

model = nn.Sequential(
            nn.Linear(
                3072,  # <1>
                512,   # <2>
            ),
            nn.Tanh(),
            nn.Linear(
                512,   # <2>
                n_out, # <3>
            )
        )

In [7]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 4),
            nn.Softmax(dim=1))

In [8]:
import time
import torch
import torch.nn as nn
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 4),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 200

start=time.time()

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    


stop = time.time()
duration = stop - start
print("This took: %f" % duration)


Epoch: 0, Loss: 0.676218
Epoch: 1, Loss: 0.776961
Epoch: 2, Loss: 0.853374
Epoch: 3, Loss: 0.564755
Epoch: 4, Loss: 0.803344
Epoch: 5, Loss: 0.488497
Epoch: 6, Loss: 0.756503
Epoch: 7, Loss: 0.591394
Epoch: 8, Loss: 0.346054
Epoch: 9, Loss: 0.681335
Epoch: 10, Loss: 0.506724
Epoch: 11, Loss: 0.431442
Epoch: 12, Loss: 0.326245
Epoch: 13, Loss: 0.657909
Epoch: 14, Loss: 0.335618
Epoch: 15, Loss: 0.362380
Epoch: 16, Loss: 0.474898
Epoch: 17, Loss: 0.301787
Epoch: 18, Loss: 0.408509
Epoch: 19, Loss: 0.420671
Epoch: 20, Loss: 0.508929
Epoch: 21, Loss: 0.402279
Epoch: 22, Loss: 0.449600
Epoch: 23, Loss: 0.191437
Epoch: 24, Loss: 0.210836
Epoch: 25, Loss: 0.357535
Epoch: 26, Loss: 0.248501
Epoch: 27, Loss: 0.153693
Epoch: 28, Loss: 0.204425
Epoch: 29, Loss: 0.215693
Epoch: 30, Loss: 0.231962
Epoch: 31, Loss: 0.163594
Epoch: 32, Loss: 0.176141
Epoch: 33, Loss: 0.145441
Epoch: 34, Loss: 0.077772
Epoch: 35, Loss: 0.168725
Epoch: 36, Loss: 0.047274
Epoch: 37, Loss: 0.155210
Epoch: 38, Loss: 0.103

In [9]:
numel_list = [p.numel()
              for p in model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(1575428, [1572864, 512, 2048, 4])

In [10]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 1.000000


In [11]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.775750


In [12]:
model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 4))

loss_fn = nn.CrossEntropyLoss()

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 4))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 200

start=time.time()

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    
stop = time.time()
duration = stop - start
print("This took: %f" % duration)

Epoch: 0, Loss: 0.876239
Epoch: 1, Loss: 0.497143
Epoch: 2, Loss: 0.940165
Epoch: 3, Loss: 0.753213
Epoch: 4, Loss: 0.696016
Epoch: 5, Loss: 0.575973
Epoch: 6, Loss: 0.540195
Epoch: 7, Loss: 0.689231
Epoch: 8, Loss: 0.723119
Epoch: 9, Loss: 0.491099
Epoch: 10, Loss: 0.630736
Epoch: 11, Loss: 0.242368
Epoch: 12, Loss: 0.486530
Epoch: 13, Loss: 0.364790
Epoch: 14, Loss: 0.490293
Epoch: 15, Loss: 0.863122
Epoch: 16, Loss: 0.367677
Epoch: 17, Loss: 0.632704
Epoch: 18, Loss: 0.344820
Epoch: 19, Loss: 0.426928
Epoch: 20, Loss: 0.181219
Epoch: 21, Loss: 0.365319
Epoch: 22, Loss: 0.147464
Epoch: 23, Loss: 0.246881
Epoch: 24, Loss: 0.295027
Epoch: 25, Loss: 0.264498
Epoch: 26, Loss: 0.151768
Epoch: 27, Loss: 0.198346
Epoch: 28, Loss: 0.111343
Epoch: 29, Loss: 0.119543
Epoch: 30, Loss: 0.056430
Epoch: 31, Loss: 0.185058
Epoch: 32, Loss: 0.075985
Epoch: 33, Loss: 0.098311
Epoch: 34, Loss: 0.029647
Epoch: 35, Loss: 0.484218
Epoch: 36, Loss: 0.028288
Epoch: 37, Loss: 0.015729
Epoch: 38, Loss: 0.006

In [14]:
numel_list = [p.numel()
              for p in model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(3737732, [3145728, 1024, 524288, 512, 65536, 128, 512, 4])

In [15]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 1.000000


In [16]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.762750
